<a href="https://colab.research.google.com/github/GabrielPlaza95/T1-Redes-Neurais/blob/main/Trabalho%201%20Redes%20Neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/GabrielPlaza95/T1-Redes-Neurais.git

%cd T1-Redes-Neurais

fatal: destination path 'T1-Redes-Neurais' already exists and is not an empty directory.


In [15]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
